In [1]:
import numpy as np
import math
import pprint

In [2]:
d = {
    'common' : {
        'region'                 : 6,    # 地域の区分
        'structure'              : 'RC', # 建物の構造
        'height'                 : 9.1,  # 建物の高さ(m)
        'opening_main_direction' : 'S',  # 主開口部の方位
    },
    'private_area' : {
        'private_floors' : [    # 各階の情報
            {
                'number_of_houses'    : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'peri_length_inside'  : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'peri_length_private' : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.48,     # 窓面積の合計(m2)
                'roof_area'           : 0.0,       # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : None,      # (最下階を除く)外気床の面積の合計(m2)
            },
            {
                'number_of_houses'    : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'peri_length_inside'  : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'peri_length_private' : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.64,     # 窓面積の合計(m2)
                'roof_area'           : 0.0,       # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : 0.0,       # (最下階を除く)外気床の面積の合計(m2)
            },
            {
                'number_of_houses'    : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'peri_length_inside'  : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'peri_length_private' : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.64,     # 窓面積の合計(m2)
                'roof_area'           : None,      # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : 0.0,       # (最下階を除く)外気床の面積の合計(m2)
            },
        ],
        'wall' : {                 # 外壁の仕様
            'U_value' : 0.785,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'inside',  # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'roof' : {                 # 屋根の仕様
            'U_value' : 0.417,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'floor' : {                # 最下階床の仕様
            'U_value' : 0.507,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'null'     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'inside_wall' : {
            'U_value'   : 2.34, # 界壁の熱貫流率(U値)(W/m2K)
        },
        'inside_floor' : {
            'U_value'  : 1.796, # 界床の熱貫流率(U値)(W/m2K)
        },
        'window' : {
            'U_value'   : 4.65, # 窓の熱貫流率(U値)(W/m2K)
            'eta_value' : 0.79, # 窓の日射熱取得率(η値)    
        }
    },
}

In [3]:
def get_floor_positions(floor_indices, number_of_floors):
    
    def func(floor_index):
        
        if floor_index == 0:
            return 'bottom'
        elif floor_index == number_of_floors-1:
            return 'top'
        else:
            return 'middle'
    
    return [func(floor_index) for floor_index in floor_indices]

In [4]:
def get_floor_heights(H_total, n):
    
    return np.full(n, H_total/n)

In [5]:
def get_roof_area_totals(A_r_total_inputs, A_f_totals, floor_positions):
    
    def func(A_r_total_input, A_f_total, floor_position):
        
        if floor_position == 'top':
            return A_f_total
        else:
            return A_r_total_input
    
    return [func(A_r_total_input, A_f_total, floor_position)
            for (A_r_total_input, A_f_total, floor_position) in zip(A_r_total_inputs, A_f_totals, floor_positions)]

In [6]:
def get_outside_floor_area_totals(A_of_total_inputs, floor_positions):
    
    def func(A_of_total_input, floor_position):
        
        if floor_position == 'bottom':
            return 0.0
        else:
            return A_of_total_input
    
    return [func(A_of_total_input, floor_position)
            for (A_of_total_input, floor_position) in zip(A_of_total_inputs, floor_positions)]

In [7]:
def make_private_floors(h, fs):
    
    # 床面積の合計, m2, [1次元配列]
    floor_area_totals = np.array([f['floor_area'] for f in fs])
    print('床面積の合計:' + str(floor_area_totals))
    
    # 屋根の面積の合計(入力値), m2, [1次元配列]
    roof_area_total_inputs = np.array([f['roof_area'] for f in fs])
    print('屋根の面積の合計(入力値):' + str(roof_area_total_inputs))
    
    # 外気床の面積の合計(入力値), m2, [1次元配列]
    outside_floor_area_inputs = np.array([f['outside_floor_area'] for f in fs])
    print('外気床の面積の合計(入力値):' + str(outside_floor_area_inputs))

    # 階数
    number_of_floors = len(fs)
    print('階数:' + str(number_of_floors))
    
    # 階のindex(0はじまり) [1次元配列]
    floor_indices = np.arange(number_of_floors)
    print('階index:' + str(floor_indices))
    
    # フロアの種類(最上階・中間階・最下階) [1次元配列]
    floor_positions = get_floor_positions(floor_indices, number_of_floors)
    print('フロアの種類:' + str(floor_positions))
    
    # 各フロアの階高, m, [1次元配列]
    floor_heights = get_floor_heights(h, number_of_floors)
    print('各フロアの階高:' + str(floor_heights))
    
    # 屋根面積の合計, m2, [1次元配列]
    roof_area_totals = get_roof_area_totals(roof_area_total_inputs, floor_area_totals, floor_positions)
    print('屋根面積の合計:' + str(roof_area_totals))
    
    # 外気床面積の合計, m2, [1次元配列]
    outside_floor_area_totals = get_outside_floor_area_totals(outside_floor_area_inputs, floor_positions)
    print('外気床面積の合計:' + str(outside_floor_area_totals))
    
    number_of_houses = [f['number_of_houses'] for f in fs]
    
    return []

In [8]:
def make_private_area(height, wall, roof, floor, inside_wall, inside_floor, window, private_floors):
    
    fs = make_private_floors(h=height, fs=private_floors)

    return {
        'wall' : wall,
        'roof' : roof,
        'floor' : floor,
        'inside_wall' : inside_wall,
        'inside_floor' : inside_floor,
        'window' : window,
        'private_floors' : fs,
    }

In [9]:
def make_common(region, structure, height, opening_main_direction):
    
    return {
        'region' : region,
        'structure' : structure,
        'height' : height,
        'opening_main_direction' : opening_main_direction,
    }

In [10]:
def convert(d):
    
    common = make_common(**d['common'])
    private_area = make_private_area(common['height'], **d['private_area'])
    return {
        'common' : common,
        'private_area' : private_area
    }

In [11]:
convert(d)

床面積の合計:[162.64 162.64 162.64]
屋根の面積の合計(入力値):[0.0 0.0 None]
外気床の面積の合計(入力値):[None 0.0 0.0]
階数:3
階index:[0 1 2]
フロアの種類:['bottom', 'middle', 'top']
各フロアの階高:[3.03333333 3.03333333 3.03333333]
屋根面積の合計:[0.0, 0.0, 162.64]
外気床面積の合計:[0.0, 0.0, 0.0]


{'common': {'region': 6,
  'structure': 'RC',
  'height': 9.1,
  'opening_main_direction': 'S'},
 'private_area': {'wall': {'U_value': 0.785,
   'ins_pos': 'inside',
   'HB_spec': 'spec1'},
  'roof': {'U_value': 0.417, 'ins_pos': 'outside', 'HB_spec': 'spec1'},
  'floor': {'U_value': 0.507, 'ins_pos': 'outside', 'HB_spec': 'null'},
  'inside_wall': {'U_value': 2.34},
  'inside_floor': {'U_value': 1.796},
  'window': {'U_value': 4.65, 'eta_value': 0.79},
  'private_floors': []}}